# Interroger des documents (RAG)

In [ ]:
%pip install -qU wget openai

In [2]:
import os
import requests
from openai import OpenAI
import wget

In [3]:
# OpenAI client configuration
base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = "YOUR_API_KEY"

client = OpenAI(base_url=base_url, api_key=api_key)

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}

Commençons par télécharger le document que nous souhaitons interroger. Ce document peut être un pdf, un fichier html ou un fichier json.


In [5]:
# Download a file
file_path = "my_document.pdf"
if not os.path.exists(file_path):
    doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"
    wget.download(doc_url, out=file_path)


Pour commencer, nous créons une collection nommée `tutorial`. Pour cela nous effectuons une requête GET sur l'endpoint `/v1/models` afin d'obtenir la liste des modèles disponibles et définissons le modèle d'embeddings à utiliser.

Nous allons avoir besoin également d'un modèle de langage. Nous appelons le endpoint `/v1/models` pour obtenir la liste des modèles. Les modèles de langage ont le type *text-generation* et les modèles d'embeddings le type *text-embeddings-inference*.


In [8]:
language_model, embeddings_model = None, None

for model in client.models.list().data:
    if model.type == "text-generation" and language_model is None:
        language_model = model.id
    if model.type == "text-embeddings-inference" and embeddings_model is None:
        embeddings_model = model.id

print(f"language model: {language_model}\nembeddings model: {embeddings_model}")

language model: AgentPublic/llama3-instruct-8b
embeddings model: BAAI/bge-m3


In [7]:
collection = "tutorial"

response = session.post(f"{base_url}/collections", json={"name": collection, "model": embeddings_model})
response = response.json()
collection_id = response["id"]
print(f"Collection ID: {collection_id}")

Collection ID: 6c6dd988-bd3d-4449-bfdd-106cda5d22ad


Enfin pour nous importons le document dans la collection de notre base vectorielle à l'aide du endpoint POST `/v1/files`.

In [8]:
files = {"file": (os.path.basename(file_path), open(file_path, "rb"), "application/pdf")}
data = {"request": '{"collection": "%s"}' % collection_id}
response = session.post(f"{base_url}/files", data=data, files=files)
assert response.status_code == 201

<Response [201]>


Nous pouvons observer que le fichier que nous avons importé est bien dans la collection à l'aide du endpoint GET `/v1/documents`.

In [9]:
response = session.get(f"{base_url}/documents/{collection_id}")
assert response.status_code == 200
files = response.json()["data"]
print(f"Number of files in collection: {len(files)}")

Number of files in collection: 2


Maintenant que nous avons notre collection et notre fichier, nous pouvons faire une recherche vectorielle à l'aide du endpoint POST `/v1/search`. Ces résutats de recherche vectorielle seront utilisés pour générer une réponse à l'aide du modèle de langage.

## Les méthodes de recherche

Trois méthodes de recherche sont disponibles :
- lexicale
- sémantique (méthode par défault)
- hybride 

### Lexicale

La méthode lexicale est la plus simple. Elle ne fait pas de recherche vectorielle mais se base uniquement sur la similarité lexicale entre la question et le contenu des documents à l'aide de l'algorithme [BM25](https://en.wikipedia.org/wiki/Okapi_BM25).


In [14]:
prompt = "Qui est Ulrich Tan ?"
data = {"collections": [collection_id], "k": 6, "prompt": prompt, "method": "lexical"}
response = session.post(url=f"{base_url}/search", json=data, headers={"Authorization": f"Bearer {api_key}"})

prompt_template = "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}\n\nDocuments :\n\n{chunks}"
chunks = "\n\n\n".join([result["chunk"]["content"] for result in response.json()["data"]])
sources = set([result["chunk"]["metadata"]["document_name"] for result in response.json()["data"]])
rag_prompt = prompt_template.format(prompt=prompt, chunks=chunks)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=language_model,
    stream=False,
    n=1,
)

response = response.choices[0].message.content
print(response)

Selon les documents, Ulrich Tan est le chef du pôle Datamin du département "Étalab".


## Sémantique (méthode par défaut)

La méthode sémantique se base sur la similarité vectorielle (similarité cosinus) entre la question et la représentation vectorielle des documents.

In [ ]:
prompt = "Qui est Ulrich Tan ?"
data = {"collections": [collection_id], "k": 6, "prompt": prompt, "method": "semantic"}
response = session.post(url=f"{base_url}/search", json=data)

prompt_template = "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}\n\nDocuments :\n\n{chunks}"
chunks = "\n\n\n".join([result["chunk"]["content"] for result in response.json()["data"]])
sources = set([result["chunk"]["metadata"]["document_name"] for result in response.json()["data"]])
prompt = prompt_template.format(prompt=prompt, chunks=chunks)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=language_model,
    stream=False,
    n=1,
)

response = response.choices[0].message.content
print(response)

## Hybride

La méthode hybride est une combinaison de la méthode lexicale et de la méthode vectorielle. Elle se base sur la similarité lexicale entre la question et le contenu des documents mais également sur la similarité vectorielle entre la question et le contenu des documents. Pour plus d'informations voir [cet article](https://weaviate.io/blog/hybrid-search-explained).


In [15]:
prompt = "Qui est Ulrich Tan ?"
data = {"collections": [collection_id], "k": 6, "prompt": prompt, "method": "hybrid"}
response = session.post(url=f"{base_url}/search", json=data, headers={"Authorization": f"Bearer {api_key}"})

prompt_template = "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}\n\nDocuments :\n\n{chunks}"
chunks = "\n\n\n".join([result["chunk"]["content"] for result in response.json()["data"]])
sources = set([result["chunk"]["metadata"]["document_name"] for result in response.json()["data"]])
prompt = prompt_template.format(prompt=prompt, chunks=chunks)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=language_model,
    stream=False,
    n=1,
)

response = response.choices[0].message.content
print(response)

Selon les documents fournis, Ulrich Tan est chef du pôle Datamin du département "Étalab".


## Recherche sur internet

Vous pouvez également faire ajouter une recherche sur internet en spécifiant "internet" dans la liste des collections.

In [9]:
data = {"collections": ["internet"], "k": 6, "prompt": prompt}
response = session.post(url=f"{base_url}/search", json=data)

chunks = "\n\n\n".join([result["chunk"]["content"] for result in response.json()["data"]])
sources = set([result["chunk"]["metadata"]["document_name"] for result in response.json()["data"]])
rag_prompt = prompt_template.format(prompt=prompt, chunks=chunks)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": rag_prompt}],
    model=language_model,
    stream=False,
    n=1,
)

response = response.choices[0].message.content
print(response)

Selon les documents, Ulrich Tan est le chef du DataLab au sein de la Direction interministérielle du numérique (Dinum), où il est responsable de coordonner l'équipe du DataLab et d'accompagner les acteurs publics dans l'identification et la priorisation de cas d'usage d'intelligence artificielle pour leur administration. Il est également considéré comme un "jeune quadra génie du numérique" et a été embauché par l'État un an avant pour introduire l'intelligence artificiale à différents étages de l'administration pour la rendre plus efficace et plus rapide, à la fois pour les fonctionnaires et les citoyens.


On peut observer que les sources sont des pages web.

In [7]:
for source in sources:
    print(source)

https://www.lefigaro.fr/conjoncture/ulrich-tan-cet-ingenieur-qui-introduit-l-ia-dans-les-administrations-pour-les-rendre-plus-efficaces-20240422
https://www.etalab.gouv.fr/datalab/equipe/
